# Stock_Screener (GUPPY)

What is a Stock Screener?

A stock screener is a tool to shortlist few companies from a pool of all the listed companies on a stock exchange using filters on the basis of indicator result (RSI,GUPPY,SMA,).shows the result of buy and sell stocks, which one is better option for buy or sell .                       
The investors specify the filters and the stock screener gives the results accordingly.
Stock screeners are very useful as it can save you a lot of time.      
You do not need to go through all the listed companies to shortlist few good ones. You can just apply the basic filter to get the list of few good ones that you want to investigate further.
Overall, the stock screener will help you to find good performing stocks according to your specifications with a single click.

 * Import All libraries required for BOT

In [ ]:
import time
import secrets 
import threading
import webbrowser
import numpy as np
import pandas as pd
from pytz import timezone
from math import modf
from pprint import pprint
from datetime import datetime
from time import gmtime, strftime
from kiteconnect import KiteTicker 
from kiteconnect import KiteConnect
from IPython.display import clear_output

# LOGIN TO ZERODHA

* Enter api key & api secret

In [ ]:
# Enter Api key and api sectet 
api_key='******************* @@enter your api key here@@@**************' 
api_secret='**********************@@enter your api key here****************'

webbrowser.open_new_tab(KiteConnect(api_key,api_secret).login_url())
kite = KiteConnect(api_key=api_key)
data = kite.generate_session(((str(input("ENTER full link generated in URL :- ")).split("request_token=")[1]).split("&action"))[0],api_secret)
kite.set_access_token(data["access_token"])
print(" \n ")
display(data)
print("  \t \t \t \t \t  WELCOME TO BACKTESTING ")
print("\n" )

# Implementation of Coding

In [29]:
print("  \t \t \t \t \t \n WELCOME TO SCREENER ")
print("\n" )
scan=str(input("DO you want Scanning ? YES/NO :-"))
if "YES"==scan or "yes"==scan:
    print("SCANNING START")
    #z=list(pd.read_csv("list.csv")["Symbol"])
    z=['ADANIPORTS', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BPCL', 'BHARTIARTL', 'INFRATEL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DRREDDY', 'EICHERMOT', 'GAIL', 'GRASIM', 'HCLTECH', 'HDFCBANK', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'HDFC', 'ICICIBANK', 'ITC', 'IBULHSGFIN', 'IOC', 'INDUSINDBK', 'INFY', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NTPC', 'ONGC', 'POWERGRID', 'RELIANCE', 'SBIN', 'SUNPHARMA', 'TCS', 'TATAMOTORS', 'TATASTEEL', 'TECHM', 'TITAN', 'UPL', 'ULTRACEMCO', 'VEDL', 'WIPRO', 'YESBANK', 'ZEEL']
    eexchange="NSE"
    time_frame     ="5minute" ###
    sdate          ="2019-07-02"
    todate         ="2019-10-02"
    tokenall=[]
    aa=0
    print("  \t \t \t \n Getting All tokens for processing BUY SELL ")
    while(True):
        ttoken=int(pd.DataFrame(kite.ltp(eexchange+":"+z[aa])).iloc[-2,0])
        tokenall.append(ttoken)
        aa=aa+1
        if aa==50:
            print("  \t \t \t \n Complete ! All tokens are fetched from file ")
            print("\n" )
            print(tokenall)
            break
    print(" Now checking Condition of BUY sell of GUPPY ")
    #Variables
    eduu=[]
    eduu2=[]
    eduu3=[]
    buy5minute=[]
    sell5minute=[]
    buy10minute=[]
    sell10minute=[]
    buy15minute=[]
    sell15minute=[]
    ##
    lst_candle=[]
    lst_heikin_nor=[]
    lst_heikin=[]
    lst_cand=[]
    lst_c=[]
    anchor=0
    countstart=0
    #programe start
    a=0
    BUY_listindicator=[]
    SELL_listindicator=[]
    sell5minutesym=[]
    buy5minutesym=[]
    buy10minutesym=[]
    sell10minutesym=[]
    buy15minutesym=[]
    sell15minutesym=[]
    #
    #price
    price5min_buy=[]
    price5min_sell=[]
    price15min_buy=[]
    price15min_sell=[]
    price30min_buy=[]
    price30min_sell=[]
    priceedit=[]
    print(a)
    def ashi():
        global a
        global BUY_listindicator
        global SELL_listindicator
        while(True):
                print("\n" )
                print("Current Token Number which is processing is",a)
            #km=datetime.now().minute
            #ks=datetime.now().second
            #if km%1==0 and ks==1:
                clear_output(wait=True)  
                now_utc = datetime.now(timezone('UTC'))
                now_asia = now_utc.astimezone(timezone('Asia/Kolkata'))
                #now_asia = now_asia.strftime("%S ")
                now_asia = now_asia.strftime("%Y-%m-%d _ %H:%M:%S ")
                edu=now_asia
                eduu.append(edu)
                dff=kite.historical_data(tokenall[a],sdate,todate,time_frame,0)
                dfw=pd.DataFrame(dff)[:-1]
                df=pd.DataFrame(dfw[['date','open','high','low','close']])
                slow_ema = [3,5,7,9,11,13,15,17,19,21,23]
                fast_ema = [25,28,31,34,37,40,43,46,49,52,55,58,61,64,67,70,200]
                def EMA(df, base, target, period, alpha=False):
                    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
                    if (alpha == True):
                        # (1 - alpha) * previous_val + alpha * current_val where alpha = 1 / period
                        df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
                    else:
                        # ((current_val - previous_val) * coeff) + previous_val where coeff = 2 / (period + 1)
                        df[target] = con.ewm(span=period, adjust=False).mean()
                    df.fillna(0,inplace = True)
                #     return df
                for j in slow_ema:
                    val = "ema"+"_"+str(j)
                    EMA(df,"close",val,j)
                for k in fast_ema:
                    val = "ema"+"_"+str(k)
                    EMA(df,"close",val,k)
                def super_guppy(interval,df,anchor=0):
                    anchor = 0
                    ShowBreak = True
                    ShowSwing = True
                    ShowCon = False
                    uOCCswing = False
                    Lookback = 6
                    emaFilter = False
                    mult = 0
                    buybreak = 0
                    sellbreak = 0
                    buy_barssince_var = 0
                    sell_barssince_var = 0
                    buybreak_barssince_var = 0
                    sellbreak_barssince_var = 0
                    barssince_lst = list()
                    barssince_var = 0
                    bar_count_var = 0
                    buy1 = list()
                    sell1 = list()
                    buy2 = list()
                    sell2 = list()
                    buybreak1 = list()
                    sellbreak1 = list()
                    def barssince(b,barssince_var):
                        barssince_lst = []
                        barssince_var = 0 
                        new_var = len(b)
                        for i in b[::-1]:
                            if i == 1:
                                break
                            barssince_lst.append(i)
                        barssince_var = len(barssince_lst)
                        return barssince_var
                        barssince_lst.clear()
                    #isIntraday
                    if interval < 1441 :
                        if (anchor==0 or interval <= 0 or interval >= anchor or anchor > 1441 ):
                            mult = 1
                        else:
                            if round(anchor/interval) > 1:
                                mult = round(anchor/interval)
                            else:
                                mult = 1
                    else:
                        mult = 1
                    #isIntraday Not
                    if interval > 1441:
                        if (anchor==0 or interval <= 0 or interval >= anchor or anchor < 52 ):
                            mult = mult
                        else:
                            if round(anchor/interval) > 1:
                                mult = round(anchor/interval)
                            else:
                                mult = 1
                    else:
                        mult = mult
                    mult = 1
                    for i in range(len(df)):
                        emaF1 = df.loc[i,'ema_3']
                        emaF2 = df.loc[i,'ema_5']
                        emaF3 = df.loc[i,'ema_7']
                        emaF4 = df.loc[i,'ema_9']
                        emaF5 = df.loc[i,'ema_11']
                        emaF6 = df.loc[i,'ema_13']
                        emaF7 = df.loc[i,'ema_15']
                        emaF8 = df.loc[i,'ema_17']
                        emaF9 = df.loc[i,'ema_19']
                        emaF10 = df.loc[i,'ema_21']
                        emaF11 = df.loc[i,'ema_23']
                        emaS1 = df.loc[i,'ema_25']
                        emaS2 = df.loc[i,'ema_28']
                        emaS3 = df.loc[i,'ema_31']
                        emaS4 = df.loc[i,'ema_34']
                        emaS5 = df.loc[i,'ema_37']
                        emaS6 = df.loc[i,'ema_40']
                        emaS7 = df.loc[i,'ema_43']
                        emaS8 = df.loc[i,'ema_46']
                        emaS9 = df.loc[i,'ema_49']
                        emaS10 = df.loc[i,'ema_52']
                        emaS11 = df.loc[i,'ema_55']
                        emaS12 = df.loc[i,'ema_58']
                        emaS13 = df.loc[i,'ema_61']
                        emaS14 = df.loc[i,'ema_64']
                        emaS15 = df.loc[i,'ema_67']
                        emaS16 = df.loc[i,'ema_70']
                        ema200 = df.loc[i,'ema_200'] 
                        emafast = (emaF1 + emaF2 + emaF3 + emaF4 + emaF5 + emaF6 + emaF7 + emaF8 + emaF9 + emaF10 + emaF11)/11
                        emaslow = (emaS1 + emaS2 + emaS3 + emaS4 + emaS5 + emaS6 + emaS7 + emaS8 + emaS9 + emaS10 + emaS11 + emaS12 + emaS13 + emaS14 + emaS15 + emaS16)/16
                        #Fast EMA Color Rules
                        colfastL = (emaF1>emaF2 and emaF2>emaF3 and emaF3>emaF4 and emaF4>emaF5 and emaF5>emaF6 and emaF6>emaF7 and emaF7>emaF8 and emaF8>emaF9 and emaF9>emaF10 and emaF10>emaF11)
                        colfastS = (emaF1<emaF2 and emaF2<emaF3 and emaF3<emaF4 and emaF4<emaF5 and emaF5<emaF6 and emaF6<emaF7 and emaF7<emaF8 and emaF8<emaF9 and emaF9<emaF10 and emaF10<emaF11)
                        #Slow EMA Color Rules
                        colslowL = (emaS1>emaS2 and emaS2>emaS3 and emaS3>emaS4 and emaS4>emaS5 and emaS5>emaS6 and emaS6>emaS7 and emaS7>emaS8) and (emaS8>emaS9 and emaS9>emaS10 and emaS10>emaS11 and emaS11>emaS12 and emaS12>emaS13 and emaS13>emaS14 and emaS14>emaS15 and emaS15>emaS16)
                        colslowS = (emaS1<emaS2 and emaS2<emaS3 and emaS3<emaS4 and emaS4<emaS5 and emaS5<emaS6 and emaS6<emaS7 and emaS7<emaS8) and (emaS8<emaS9 and emaS9<emaS10 and emaS10<emaS11 and emaS11<emaS12 and emaS12<emaS13 and emaS13<emaS14 and emaS14<emaS15 and emaS15<emaS16)
                        if  emafast > emaslow and not colslowS and colfastL and (not ShowCon or colslowL) and (not emaFilter or emafast>ema200):
                            if int(buy1[-1]) > 0:
                                buy = buy1[-1] + 1
                            else:
                                buy = 1
                        else:
                            buy = 0
                        buy1.append(buy)
                        if  emafast < emaslow and not colslowL and colfastS and (not ShowCon or colslowS) and (not emaFilter or emafast<ema200):
                            if int(sell1[-1]) > 0:
                                sell = sell1[-1] + 1
                            else:
                                sell = 1
                        else:
                            sell = 0
                        sell1.append(sell)
                        #buy
                        if buy>1 and colfastL and (uOCCswing and ((df.loc[i-1,'close']<df.loc[i-1,'open']) and (df.loc[i,'close']>df.loc[i,'open']))):
                            buy3 = 1
                        else:
                            buy3 = buy
                        buy2.append(buy3)
                        #sell    
                        if sell>1 and colfastS and (uOCCswing and ((df.loc[i-1,'close']<df.loc[i-1,'open']) and (df.loc[i,'close']>df.loc[i,'open']))):
                            sell3 = 1
                        else:
                            sell3 = sell
                        sell2.append(sell3)
                        #buybreak
                        if emafast > emaslow and not colslowS and (not emaFilter or emafast>ema200):
                            if buybreak1[-1] > 0:
                                buybreak = buybreak1[-1] + 1
                            else:
                                buybreak = 1
                        else:
                            buybreak = 0
                        buybreak1.append(buybreak)
                        if emafast < emaslow and not colslowL and (not emaFilter or emafast<ema200):
                            if sellbreak1[-1] > 0:
                                sellbreak = sellbreak1[-1]+1
                            else:
                                sellbreak = 1
                        else:
                            sellbreak = 0
                        sellbreak1.append(sellbreak)
                        #arrow plotting
                        #buy_arrow
                        buy_barssince_var = barssince(buy2[:-1],barssince_var)
                        if (ShowSwing and buy3==1)and buy_barssince_var > 6:
                            buy_arrow = 1
                        else:
                            buy_arrow = 0
                        #sell arrow
                        sell_barssince_var = barssince(sell2[:-1],barssince_var)
                        if ShowSwing and (sell3==1 and sell_barssince_var > 6):
                            sell_arrow = 1
                        else:
                            sell_arrow = 0
                        #buybreak_arrow
                        buybreak_barssince_var = barssince(buybreak1[:-1],barssince_var)
                        sellbreak_barssince_var = barssince(sellbreak1[:-1],barssince_var)

                        if ShowBreak and buybreak==1 and (sellbreak_barssince_var>Lookback) and (buybreak_barssince_var>Lookback):
                            buybreak_arrow = 1
                        else:
                            buybreak_arrow = 0
                        #sellbreak_arrow
                        if ShowBreak and sellbreak==1 and (buybreak_barssince_var>Lookback) and (sellbreak_barssince_var>Lookback):
                            sellbreak_arrow = 1
                        else:
                            sellbreak_arrow = 0
                        if buy_arrow==1 and sell_arrow==0 and buybreak_arrow==0 and sellbreak_arrow==0:
                            arrow_color = 'green'
                        elif buy_arrow==0 and sell_arrow==1 and buybreak_arrow==0 and sellbreak_arrow==0:
                            arrow_color = 'red'
                        elif sell_arrow==0 and (buy_arrow==0 or buy_arrow==1) and buybreak_arrow==1 and sellbreak_arrow==0:
                            arrow_color = 'aqua'
                        elif buy_arrow==0 and (sell_arrow==1 or sell_arrow==0) and buybreak_arrow==0 and sellbreak_arrow==1:
                            arrow_color = 'blue'
                        else:
                            arrow_color = 'none'
                        df.loc[i,'arrow_color'] = arrow_color
                    df = df[['date','open','high','low','close','arrow_color']]
                    return df
                df=super_guppy(15,df)
                gup=df
                print(" \n ")
                print(" \t \t \t \t Zerodha GUPPY SCREENER on 5 minute Data ")
                print(" \t \t \t \t 5 minute Program Start time is  ", eduu[0])
                print("\t \t \t \n Current Token checking  " , tokenall[a])
                print("\n" )
                print(" \t \t \t \n Current Colour on this token is " , gup.iloc[-1,5])
                if "green" in gup.iloc[-1,5]:
                    print(" BUY stock found ")
                    buy5minute.append((tokenall[a]))
                    buy5minutesym.append((z[a]))
                    price5min_buy.append(gup.iloc[-1,2])
                if "red" in gup.iloc[-1,5]:
                    print(" SELL stock found ")
                    sell5minute.append((tokenall[a]))
                    sell5minutesym.append((z[a]))
                    price5min_sell.append(gup.iloc[-1,2])
                else:
                    pass
                print("Buy stock found for 5 minute are  :="  ,buy5minute)
                print("Sell stocks found for 5 minute are:="  ,sell5minute)
                a=a+1
                if a==len(tokenall):
                    file=str(buy5minute)
                    filee=str(sell5minute)
                    with open("tokens.txt","w") as  f:
                        f.write(file)
                        f.write(filee)
                    break
                buyframe={"SYMBOLS_BUY":buy5minute,"TOKENS_BUY":buy5minutesym,"Price":price5min_buy}
                fivemin=pd.DataFrame(buyframe)
                display(fivemin)
                buyframee={"SYMBOLS_SELL":sell5minutesym,"TOKENS_SELL":sell5minute,"Price":price5min_sell}
                fivemine=pd.DataFrame(buyframee)
                display(fivemine)
    ashi()
    print("5 minute data complete! Now getting Data of 15 minutes ")
    time_frame     ="15minute"
    a=0
    def ashi15():
        global a
        global BUY_listindicator
        global SELL_listindicator
        while(True):
                print("\n" )
                print("Current Token Number which is processing is",a)
            #km=datetime.now().minute
            #ks=datetime.now().second
            #if km%1==0 and ks==1:
                clear_output(wait=True)  
                now_utc = datetime.now(timezone('UTC'))
                now_asia = now_utc.astimezone(timezone('Asia/Kolkata'))
                #now_asia = now_asia.strftime("%S ")
                now_asia = now_asia.strftime("%Y-%m-%d _ %H:%M:%S ")
                edu2=now_asia
                eduu2.append(edu2)
                dff=kite.historical_data(tokenall[a],sdate,todate,time_frame,0)
                dfw=pd.DataFrame(dff)[:-1]
                df=pd.DataFrame(dfw[['date','open','high','low','close']])
                slow_ema = [3,5,7,9,11,13,15,17,19,21,23]
                fast_ema = [25,28,31,34,37,40,43,46,49,52,55,58,61,64,67,70,200]
                def EMA(df, base, target, period, alpha=False):
                    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
                    if (alpha == True):
                        # (1 - alpha) * previous_val + alpha * current_val where alpha = 1 / period
                        df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
                    else:
                        # ((current_val - previous_val) * coeff) + previous_val where coeff = 2 / (period + 1)
                        df[target] = con.ewm(span=period, adjust=False).mean()
                    df.fillna(0,inplace = True)
                #     return df
                for j in slow_ema:
                    val = "ema"+"_"+str(j)
                    EMA(df,"close",val,j)
                for k in fast_ema:
                    val = "ema"+"_"+str(k)
                    EMA(df,"close",val,k)
                def super_guppy(interval,df,anchor=0):
                    anchor = 0
                    ShowBreak = True
                    ShowSwing = True
                    ShowCon = False
                    uOCCswing = False
                    Lookback = 6
                    emaFilter = False
                    mult = 0
                    buybreak = 0
                    sellbreak = 0
                    buy_barssince_var = 0
                    sell_barssince_var = 0
                    buybreak_barssince_var = 0
                    sellbreak_barssince_var = 0
                    barssince_lst = list()
                    barssince_var = 0
                    bar_count_var = 0
                    buy1 = list()
                    sell1 = list()
                    buy2 = list()
                    sell2 = list()
                    buybreak1 = list()
                    sellbreak1 = list()
                    def barssince(b,barssince_var):
                        barssince_lst = []
                        barssince_var = 0 
                        new_var = len(b)
                        for i in b[::-1]:
                            if i == 1:
                                break
                            barssince_lst.append(i)
                        barssince_var = len(barssince_lst)
                        return barssince_var
                        barssince_lst.clear()
                    #isIntraday
                    if interval < 1441 :
                        if (anchor==0 or interval <= 0 or interval >= anchor or anchor > 1441 ):
                            mult = 1
                        else:
                            if round(anchor/interval) > 1:
                                mult = round(anchor/interval)
                            else:
                                mult = 1
                    else:
                        mult = 1
                    #isIntraday Not
                    if interval > 1441:
                        if (anchor==0 or interval <= 0 or interval >= anchor or anchor < 52 ):
                            mult = mult
                        else:
                            if round(anchor/interval) > 1:
                                mult = round(anchor/interval)
                            else:
                                mult = 1
                    else:
                        mult = mult
                    mult = 1
                    for i in range(len(df)):
                        emaF1 = df.loc[i,'ema_3']
                        emaF2 = df.loc[i,'ema_5']
                        emaF3 = df.loc[i,'ema_7']
                        emaF4 = df.loc[i,'ema_9']
                        emaF5 = df.loc[i,'ema_11']
                        emaF6 = df.loc[i,'ema_13']
                        emaF7 = df.loc[i,'ema_15']
                        emaF8 = df.loc[i,'ema_17']
                        emaF9 = df.loc[i,'ema_19']
                        emaF10 = df.loc[i,'ema_21']
                        emaF11 = df.loc[i,'ema_23']
                        emaS1 = df.loc[i,'ema_25']
                        emaS2 = df.loc[i,'ema_28']
                        emaS3 = df.loc[i,'ema_31']
                        emaS4 = df.loc[i,'ema_34']
                        emaS5 = df.loc[i,'ema_37']
                        emaS6 = df.loc[i,'ema_40']
                        emaS7 = df.loc[i,'ema_43']
                        emaS8 = df.loc[i,'ema_46']
                        emaS9 = df.loc[i,'ema_49']
                        emaS10 = df.loc[i,'ema_52']
                        emaS11 = df.loc[i,'ema_55']
                        emaS12 = df.loc[i,'ema_58']
                        emaS13 = df.loc[i,'ema_61']
                        emaS14 = df.loc[i,'ema_64']
                        emaS15 = df.loc[i,'ema_67']
                        emaS16 = df.loc[i,'ema_70']
                        ema200 = df.loc[i,'ema_200'] 
                        emafast = (emaF1 + emaF2 + emaF3 + emaF4 + emaF5 + emaF6 + emaF7 + emaF8 + emaF9 + emaF10 + emaF11)/11
                        emaslow = (emaS1 + emaS2 + emaS3 + emaS4 + emaS5 + emaS6 + emaS7 + emaS8 + emaS9 + emaS10 + emaS11 + emaS12 + emaS13 + emaS14 + emaS15 + emaS16)/16
                        #Fast EMA Color Rules
                        colfastL = (emaF1>emaF2 and emaF2>emaF3 and emaF3>emaF4 and emaF4>emaF5 and emaF5>emaF6 and emaF6>emaF7 and emaF7>emaF8 and emaF8>emaF9 and emaF9>emaF10 and emaF10>emaF11)
                        colfastS = (emaF1<emaF2 and emaF2<emaF3 and emaF3<emaF4 and emaF4<emaF5 and emaF5<emaF6 and emaF6<emaF7 and emaF7<emaF8 and emaF8<emaF9 and emaF9<emaF10 and emaF10<emaF11)
                        #Slow EMA Color Rules
                        colslowL = (emaS1>emaS2 and emaS2>emaS3 and emaS3>emaS4 and emaS4>emaS5 and emaS5>emaS6 and emaS6>emaS7 and emaS7>emaS8) and (emaS8>emaS9 and emaS9>emaS10 and emaS10>emaS11 and emaS11>emaS12 and emaS12>emaS13 and emaS13>emaS14 and emaS14>emaS15 and emaS15>emaS16)
                        colslowS = (emaS1<emaS2 and emaS2<emaS3 and emaS3<emaS4 and emaS4<emaS5 and emaS5<emaS6 and emaS6<emaS7 and emaS7<emaS8) and (emaS8<emaS9 and emaS9<emaS10 and emaS10<emaS11 and emaS11<emaS12 and emaS12<emaS13 and emaS13<emaS14 and emaS14<emaS15 and emaS15<emaS16)
                        if  emafast > emaslow and not colslowS and colfastL and (not ShowCon or colslowL) and (not emaFilter or emafast>ema200):
                            if int(buy1[-1]) > 0:
                                buy = buy1[-1] + 1
                            else:
                                buy = 1
                        else:
                            buy = 0
                        buy1.append(buy)
                        if  emafast < emaslow and not colslowL and colfastS and (not ShowCon or colslowS) and (not emaFilter or emafast<ema200):
                            if int(sell1[-1]) > 0:
                                sell = sell1[-1] + 1
                            else:
                                sell = 1
                        else:
                            sell = 0
                        sell1.append(sell)
                        #buy
                        if buy>1 and colfastL and (uOCCswing and ((df.loc[i-1,'close']<df.loc[i-1,'open']) and (df.loc[i,'close']>df.loc[i,'open']))):
                            buy3 = 1
                        else:
                            buy3 = buy
                        buy2.append(buy3)
                        #sell    
                        if sell>1 and colfastS and (uOCCswing and ((df.loc[i-1,'close']<df.loc[i-1,'open']) and (df.loc[i,'close']>df.loc[i,'open']))):
                            sell3 = 1
                        else:
                            sell3 = sell
                        sell2.append(sell3)
                        #buybreak
                        if emafast > emaslow and not colslowS and (not emaFilter or emafast>ema200):
                            if buybreak1[-1] > 0:
                                buybreak = buybreak1[-1] + 1
                            else:
                                buybreak = 1
                        else:
                            buybreak = 0
                        buybreak1.append(buybreak)
                        if emafast < emaslow and not colslowL and (not emaFilter or emafast<ema200):
                            if sellbreak1[-1] > 0:
                                sellbreak = sellbreak1[-1]+1
                            else:
                                sellbreak = 1
                        else:
                            sellbreak = 0
                        sellbreak1.append(sellbreak)
                        #arrow plotting
                        #buy_arrow
                        buy_barssince_var = barssince(buy2[:-1],barssince_var)
                        if (ShowSwing and buy3==1)and buy_barssince_var > 6:
                            buy_arrow = 1
                        else:
                            buy_arrow = 0
                        #sell arrow
                        sell_barssince_var = barssince(sell2[:-1],barssince_var)
                        if ShowSwing and (sell3==1 and sell_barssince_var > 6):
                            sell_arrow = 1
                        else:
                            sell_arrow = 0
                        #buybreak_arrow
                        buybreak_barssince_var = barssince(buybreak1[:-1],barssince_var)
                        sellbreak_barssince_var = barssince(sellbreak1[:-1],barssince_var)

                        if ShowBreak and buybreak==1 and (sellbreak_barssince_var>Lookback) and (buybreak_barssince_var>Lookback):
                            buybreak_arrow = 1
                        else:
                            buybreak_arrow = 0
                        #sellbreak_arrow
                        if ShowBreak and sellbreak==1 and (buybreak_barssince_var>Lookback) and (sellbreak_barssince_var>Lookback):
                            sellbreak_arrow = 1
                        else:
                            sellbreak_arrow = 0
                        if buy_arrow==1 and sell_arrow==0 and buybreak_arrow==0 and sellbreak_arrow==0:
                            arrow_color = 'green'
                        elif buy_arrow==0 and sell_arrow==1 and buybreak_arrow==0 and sellbreak_arrow==0:
                            arrow_color = 'red'
                        elif sell_arrow==0 and (buy_arrow==0 or buy_arrow==1) and buybreak_arrow==1 and sellbreak_arrow==0:
                            arrow_color = 'aqua'
                        elif buy_arrow==0 and (sell_arrow==1 or sell_arrow==0) and buybreak_arrow==0 and sellbreak_arrow==1:
                            arrow_color = 'blue'
                        else:
                            arrow_color = 'none'
                        df.loc[i,'arrow_color'] = arrow_color
                    df = df[['date','open','high','low','close','arrow_color']]
                    return df
                df=super_guppy(15,df)
                gup=df
                print(" \n ")
                print(" \t \t \t \t Zerodha GUPPY SCREENER on 15 minute data ")
                print(" \t \t \t \t 15 minute Program Start time is  ", eduu2[0])
                print("\t \t \t \n Current Token checking  " , tokenall[a])
                print("\n" )
                print(" \t \t \t \n Current Colour on this token is " , gup.iloc[-1,5])
                if "green" in gup.iloc[-1,5]:
                    print("Buy stock found ")
                    buy10minute.append((tokenall[a]))
                    buy10minutesym.append((z[a]))
                    price15min_buy.append(gup.iloc[-1,2])
                if "red" in gup.iloc[-1,5]:
                    print(" Sell Stock found ")
                    sell10minute.append((tokenall[a]))
                    sell10minutesym.append((z[a]))
                    priceedit.append(gup.iloc[-1,2])
                else:
                    pass
                print("Buy stock found for 15 minute are  :="  ,buy10minute)
                print("Sell stocks found for 15 minute are:="  ,sell10minute)
                a=a+1
                if a==len(tokenall):
                    file=str(buy5minute)
                    filee=str(sell5minute)
                    file2=str(buy10minute)
                    filee2=str(sell10minute)
                    with open("tokens.txt","w") as  f:
                        f.write(file)
                        f.write(filee)
                        f.write(file2)
                        f.write(filee2)
                    break
                    
                buyframe15={"SYMBOLS_BUY":buy10minutesym,"TOKENS_BUY":buy10minute,"Price":price15min_buy}
                fifteenemin=pd.DataFrame(buyframe15)
                display(fifteenemin)
                buyframe156={"SYMBOLS_SELL":sell10minutesym,"TOKENS_SELL":sell10minute,"Price":priceedit}
                fifteenemin6=pd.DataFrame(buyframe156)
                display(fifteenemin6)

    ashi15()
    print("5 and 15 minute data complete now getting Data of 30 minutes of minute frame")
    time_frame     ="30minute"
    a=0
    def ashi30():
        global a
        global BUY_listindicator
        global SELL_listindicator
        while(True):
                print("\n" )
                print("Current Token Number which is processing is",a)
            #km=datetime.now().minute
            #ks=datetime.now().second
            #if km%1==0 and ks==1:
                clear_output(wait=True)  
                now_utc = datetime.now(timezone('UTC'))
                now_asia = now_utc.astimezone(timezone('Asia/Kolkata'))
                #now_asia = now_asia.strftime("%S ")
                now_asia = now_asia.strftime("%Y-%m-%d _ %H:%M:%S ")
                edu3=now_asia
                eduu3.append(edu3)
                dff=kite.historical_data(tokenall[a],sdate,todate,time_frame,0)
                dfw=pd.DataFrame(dff)[:-1]
                df=pd.DataFrame(dfw[['date','open','high','low','close']])
                slow_ema = [3,5,7,9,11,13,15,17,19,21,23]
                fast_ema = [25,28,31,34,37,40,43,46,49,52,55,58,61,64,67,70,200]
                def EMA(df, base, target, period, alpha=False):
                    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
                    if (alpha == True):
                        # (1 - alpha) * previous_val + alpha * current_val where alpha = 1 / period
                        df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
                    else:
                        # ((current_val - previous_val) * coeff) + previous_val where coeff = 2 / (period + 1)
                        df[target] = con.ewm(span=period, adjust=False).mean()
                    df.fillna(0,inplace = True)
                #     return df
                for j in slow_ema:
                    val = "ema"+"_"+str(j)
                    EMA(df,"close",val,j)
                for k in fast_ema:
                    val = "ema"+"_"+str(k)
                    EMA(df,"close",val,k)
                def super_guppy(interval,df,anchor=0):
                    anchor = 0
                    ShowBreak = True
                    ShowSwing = True
                    ShowCon = False
                    uOCCswing = False
                    Lookback = 6
                    emaFilter = False
                    mult = 0
                    buybreak = 0
                    sellbreak = 0
                    buy_barssince_var = 0
                    sell_barssince_var = 0
                    buybreak_barssince_var = 0
                    sellbreak_barssince_var = 0
                    barssince_lst = list()
                    barssince_var = 0
                    bar_count_var = 0
                    buy1 = list()
                    sell1 = list()
                    buy2 = list()
                    sell2 = list()
                    buybreak1 = list()
                    sellbreak1 = list()
                    def barssince(b,barssince_var):
                        barssince_lst = []
                        barssince_var = 0 
                        new_var = len(b)
                        for i in b[::-1]:
                            if i == 1:
                                break
                            barssince_lst.append(i)
                        barssince_var = len(barssince_lst)
                        return barssince_var
                        barssince_lst.clear()
                    #isIntraday
                    if interval < 1441 :
                        if (anchor==0 or interval <= 0 or interval >= anchor or anchor > 1441 ):
                            mult = 1
                        else:
                            if round(anchor/interval) > 1:
                                mult = round(anchor/interval)
                            else:
                                mult = 1
                    else:
                        mult = 1
                    #isIntraday Not
                    if interval > 1441:
                        if (anchor==0 or interval <= 0 or interval >= anchor or anchor < 52 ):
                            mult = mult
                        else:
                            if round(anchor/interval) > 1:
                                mult = round(anchor/interval)
                            else:
                                mult = 1
                    else:
                        mult = mult
                    mult = 1
                    for i in range(len(df)):
                        emaF1 = df.loc[i,'ema_3']
                        emaF2 = df.loc[i,'ema_5']
                        emaF3 = df.loc[i,'ema_7']
                        emaF4 = df.loc[i,'ema_9']
                        emaF5 = df.loc[i,'ema_11']
                        emaF6 = df.loc[i,'ema_13']
                        emaF7 = df.loc[i,'ema_15']
                        emaF8 = df.loc[i,'ema_17']
                        emaF9 = df.loc[i,'ema_19']
                        emaF10 = df.loc[i,'ema_21']
                        emaF11 = df.loc[i,'ema_23']
                        emaS1 = df.loc[i,'ema_25']
                        emaS2 = df.loc[i,'ema_28']
                        emaS3 = df.loc[i,'ema_31']
                        emaS4 = df.loc[i,'ema_34']
                        emaS5 = df.loc[i,'ema_37']
                        emaS6 = df.loc[i,'ema_40']
                        emaS7 = df.loc[i,'ema_43']
                        emaS8 = df.loc[i,'ema_46']
                        emaS9 = df.loc[i,'ema_49']
                        emaS10 = df.loc[i,'ema_52']
                        emaS11 = df.loc[i,'ema_55']
                        emaS12 = df.loc[i,'ema_58']
                        emaS13 = df.loc[i,'ema_61']
                        emaS14 = df.loc[i,'ema_64']
                        emaS15 = df.loc[i,'ema_67']
                        emaS16 = df.loc[i,'ema_70']
                        ema200 = df.loc[i,'ema_200'] 
                        emafast = (emaF1 + emaF2 + emaF3 + emaF4 + emaF5 + emaF6 + emaF7 + emaF8 + emaF9 + emaF10 + emaF11)/11
                        emaslow = (emaS1 + emaS2 + emaS3 + emaS4 + emaS5 + emaS6 + emaS7 + emaS8 + emaS9 + emaS10 + emaS11 + emaS12 + emaS13 + emaS14 + emaS15 + emaS16)/16
                        #Fast EMA Color Rules
                        colfastL = (emaF1>emaF2 and emaF2>emaF3 and emaF3>emaF4 and emaF4>emaF5 and emaF5>emaF6 and emaF6>emaF7 and emaF7>emaF8 and emaF8>emaF9 and emaF9>emaF10 and emaF10>emaF11)
                        colfastS = (emaF1<emaF2 and emaF2<emaF3 and emaF3<emaF4 and emaF4<emaF5 and emaF5<emaF6 and emaF6<emaF7 and emaF7<emaF8 and emaF8<emaF9 and emaF9<emaF10 and emaF10<emaF11)
                        #Slow EMA Color Rules
                        colslowL = (emaS1>emaS2 and emaS2>emaS3 and emaS3>emaS4 and emaS4>emaS5 and emaS5>emaS6 and emaS6>emaS7 and emaS7>emaS8) and (emaS8>emaS9 and emaS9>emaS10 and emaS10>emaS11 and emaS11>emaS12 and emaS12>emaS13 and emaS13>emaS14 and emaS14>emaS15 and emaS15>emaS16)
                        colslowS = (emaS1<emaS2 and emaS2<emaS3 and emaS3<emaS4 and emaS4<emaS5 and emaS5<emaS6 and emaS6<emaS7 and emaS7<emaS8) and (emaS8<emaS9 and emaS9<emaS10 and emaS10<emaS11 and emaS11<emaS12 and emaS12<emaS13 and emaS13<emaS14 and emaS14<emaS15 and emaS15<emaS16)
                        if  emafast > emaslow and not colslowS and colfastL and (not ShowCon or colslowL) and (not emaFilter or emafast>ema200):
                            if int(buy1[-1]) > 0:
                                buy = buy1[-1] + 1
                            else:
                                buy = 1
                        else:
                            buy = 0
                        buy1.append(buy)
                        if  emafast < emaslow and not colslowL and colfastS and (not ShowCon or colslowS) and (not emaFilter or emafast<ema200):
                            if int(sell1[-1]) > 0:
                                sell = sell1[-1] + 1
                            else:
                                sell = 1
                        else:
                            sell = 0
                        sell1.append(sell)
                        #buy
                        if buy>1 and colfastL and (uOCCswing and ((df.loc[i-1,'close']<df.loc[i-1,'open']) and (df.loc[i,'close']>df.loc[i,'open']))):
                            buy3 = 1
                        else:
                            buy3 = buy
                        buy2.append(buy3)
                        #sell    
                        if sell>1 and colfastS and (uOCCswing and ((df.loc[i-1,'close']<df.loc[i-1,'open']) and (df.loc[i,'close']>df.loc[i,'open']))):
                            sell3 = 1
                        else:
                            sell3 = sell
                        sell2.append(sell3)
                        #buybreak
                        if emafast > emaslow and not colslowS and (not emaFilter or emafast>ema200):
                            if buybreak1[-1] > 0:
                                buybreak = buybreak1[-1] + 1
                            else:
                                buybreak = 1
                        else:
                            buybreak = 0
                        buybreak1.append(buybreak)
                        if emafast < emaslow and not colslowL and (not emaFilter or emafast<ema200):
                            if sellbreak1[-1] > 0:
                                sellbreak = sellbreak1[-1]+1
                            else:
                                sellbreak = 1
                        else:
                            sellbreak = 0
                        sellbreak1.append(sellbreak)
                        #arrow plotting
                        #buy_arrow
                        buy_barssince_var = barssince(buy2[:-1],barssince_var)
                        if (ShowSwing and buy3==1)and buy_barssince_var > 6:
                            buy_arrow = 1
                        else:
                            buy_arrow = 0
                        #sell arrow
                        sell_barssince_var = barssince(sell2[:-1],barssince_var)
                        if ShowSwing and (sell3==1 and sell_barssince_var > 6):
                            sell_arrow = 1
                        else:
                            sell_arrow = 0
                        #buybreak_arrow
                        buybreak_barssince_var = barssince(buybreak1[:-1],barssince_var)
                        sellbreak_barssince_var = barssince(sellbreak1[:-1],barssince_var)

                        if ShowBreak and buybreak==1 and (sellbreak_barssince_var>Lookback) and (buybreak_barssince_var>Lookback):
                            buybreak_arrow = 1
                        else:
                            buybreak_arrow = 0
                        #sellbreak_arrow
                        if ShowBreak and sellbreak==1 and (buybreak_barssince_var>Lookback) and (sellbreak_barssince_var>Lookback):
                            sellbreak_arrow = 1
                        else:
                            sellbreak_arrow = 0
                        if buy_arrow==1 and sell_arrow==0 and buybreak_arrow==0 and sellbreak_arrow==0:
                            arrow_color = 'green'
                        elif buy_arrow==0 and sell_arrow==1 and buybreak_arrow==0 and sellbreak_arrow==0:
                            arrow_color = 'red'
                        elif sell_arrow==0 and (buy_arrow==0 or buy_arrow==1) and buybreak_arrow==1 and sellbreak_arrow==0:
                            arrow_color = 'aqua'
                        elif buy_arrow==0 and (sell_arrow==1 or sell_arrow==0) and buybreak_arrow==0 and sellbreak_arrow==1:
                            arrow_color = 'blue'
                        else:
                            arrow_color = 'none'
                        df.loc[i,'arrow_color'] = arrow_color
                    df = df[['date','open','high','low','close','arrow_color']]
                    return df
                df=super_guppy(15,df)
                gup=df
                print(" \n ")
                print(" \t \t \t \t Zerodha GUPPY SCREENER for 30 minute Data")
                print(" \t \t \t \t 30 minute Program Start time is  ", eduu3[0])
                print("\n" )
                print("\t \t \t \n Current Token checking  " , tokenall[a])
                print("\n" )
                print(" \t \t \t \n Current Colour on this token is " , gup.iloc[-1,5])
                if "green" in gup.iloc[-1,5]:
                    print("BUY stock found ")
                    buy15minute.append((tokenall[a]))
                    buy15minutesym.append((z[a]))
                    price30min_buy.append(gup.iloc[-1,2])
                if "red" in gup.iloc[-1,5]:
                    print(" SELL Stock found ")
                    sell15minute.append((tokenall[a]))
                    sell15minutesym.append((z[a]))
                    price30min_sell.append(gup.iloc[-1,2])
                else:
                    pass
                print("Buy stock found for 30 minute are  :="  ,buy15minute)
                print("Sell stocks found for 30 minute are:="  ,sell15minute)
                a=a+1
                if a==len(tokenall):
                    file=str(buy5minute)
                    filee=str(sell5minute)
                    file2=str(buy10minute)
                    filee2=str(sell10minute)
                    file3=str(buy15minute)
                    filee3=str(sell15minute)
                    with open("tokens","w") as  f:
                        f.write(file)
                        f.write(filee)
                        f.write(file2)
                        f.write(filee2)
                        f.write(file3)
                        f.write(filee3)
                        print("SCANNING COMPLETE")
                    break
                buyframe30={"SYMBOLS_BUY":buy15minutesym,"TOKENS_BUY":buy15minute,"Price":price30min_buy}
                thirtymin=pd.DataFrame(buyframe30)
                display(thirtymin)
                buyframe303={"SYMBOLS_SELL":sell15minutesym,"TOKENS_SELL":sell15minute,"Price":price30min_sell}
                thirtymin3=pd.DataFrame(buyframe303)
                display(thirtymin3)
    ashi30()
    print(" \n ")
    print("5 ,15 and 30 are completed ! Results are ")
    print("\n BUY/SELL 5 minute frame" )
    print(" \t \t \t \t 5 minute Program Start time is  ", eduu[0])
    print(" \n ")
    print("\n BUY/SELL 5 minute frame" )
    buyframe={"SYMBOLS_BUY":buy5minute,"TOKENS_BUY":buy5minutesym,"Price":price5min_buy}
    fivemin=pd.DataFrame(buyframe)
    display(fivemin)
    buyframee={"SYMBOLS_SELL":sell5minutesym,"TOKENS_SELL":sell5minute,"Price":price5min_sell}
    fivemine=pd.DataFrame(buyframee)
    display(fivemine)
    print("\n BUY/SELL 15 minute frame" )
    print(" \t \t \t \t 15 minute Program Start time is  ", eduu2[0])
    buyframe15={"SYMBOLS_BUY":buy10minutesym,"TOKENS_BUY":buy10minute,"Price":price15min_buy}
    fifteenemin=pd.DataFrame(buyframe15)
    display(fifteenemin)
    buyframe156={"SYMBOLS_SELL":sell10minutesym,"TOKENS_SELL":sell10minute,"Price":priceedit}
    fifteenemin6=pd.DataFrame(buyframe156)
    display(fifteenemin6)
    print("\n" )
    print("\n BUY/SELL 30 minute frame" )
    print(" \t \t \t \t 30 minute Program Start time is  ", eduu3[0])
    buyframe30={"SYMBOLS_BUY":buy15minutesym,"TOKENS_BUY":buy15minute,"Price":price30min_buy}
    thirtymin=pd.DataFrame(buyframe30)
    display(thirtymin)
    buyframe303={"SYMBOLS_SELL":sell15minutesym,"TOKENS_SELL":sell15minute,"Price":price30min_sell}
    thirtymin3=pd.DataFrame(buyframe303)
    display(thirtymin3)
    print("\n")
    # buy
    k=set(buy5minute)
    l=set(buy10minute)
    m=set(buy15minute)
    # sell
    n=set(sell5minute)
    s=set(sell10minute)
    p=set(sell15minute)
    buya=n|s|p
    sellb=k|l|m
    print("  \t \t \t \t \t ALL ' BUY | SELL '  - Neglecting Common")
    print("\n")
    print("BUY from ALL  ",sellb)
    print("SELL From ALL ",buya)
    print("\n")
    print("  \t \t \t \t \t COMMON ' BUY & SELL ' ")
    print("\n")
    buycommmon=n&s&p
    sellcommon=k&l&m
    print("BUY from ALL  ",buycommmon)
    print("SELL From ALL ",sellcommon)
    print("\n")
    # finding coomon name 
    comm_buy=[]
    comm_buy_sym=[]
    comm_sell=[]
    comm_sell_sym=[]
    ee=0
    buy_commmon=list(buycommmon)
    sell_common=list(sellcommon)
    if buy_commmon:
        print("BUY COMMON FOUND")
        while (True):
            df=pd.DataFrame(kite.instruments("NSE"))[["instrument_token","tradingsymbol","name"]]
            zall=df[df["instrument_token"]==buy_commmon[ee]]
            comm_buy.append(zall.iloc[-1,0])
            comm_buy_sym.append(zall.iloc[-1,1])
            ee=ee+1
            if ee==len(buy_commmon):
                break
    if sell_common:
        print("BUY COMMON FOUND")
        while (True):
            df=pd.DataFrame(kite.instruments("NSE"))[["instrument_token","tradingsymbol","name"]]
            zall=df[df["instrument_token"]==sell_common[ee]]
            comm_sell.append(zall.iloc[-1,0])
            comm_sell_sym.append(zall.iloc[-1,1])
            ee=ee+1
            if ee==len(sell_common):
                break
    print("BUY sell COMMON STOCK")
    dic_comm_buy={"BUY_common_symbol":comm_buy_sym, "BUY_common_Token":comm_buy}
    dic_comm_buy_disp=pd.DataFrame(dic_comm_buy)
    dic_comm_sell={"SELL_common_symbol":comm_sell_sym, "SELL_common_Token":comm_sell}
    dic_comm_sell_disp=pd.DataFrame(dic_comm_sell)
    display(dic_comm_buy_disp)
    display(dic_comm_sell_disp)
    print("\t \t \t \t \t !! ALL COMPLETED !!")
    print(" \n ")

 
 
 	 	 	 	 Zerodha GUPPY SCREENER for 30 minute Data
 	 	 	 	 30 minute Program Start time is   2019-07-25 _ 14:11:16 


	 	 	 
 Current Token checking   975873


 	 	 	 
 Current Colour on this token is  none
Buy stock found for 30 minute are  := []
Sell stocks found for 30 minute are:= [415745, 633601]
SCANNING COMPLETE
 
 
5 ,15 and 30 are completed ! Results are 

 BUY/SELL 5 minute frame
 	 	 	 	 5 minute Program Start time is   2019-07-25 _ 14:06:33 
 	 	 	 	 15 minute Program Start time is   2019-07-25 _ 14:09:54 
 	 	 	 	 30 minute Program Start time is   2019-07-25 _ 14:11:16 
 
 

 BUY/SELL 5 minute frame


Empty DataFrame
Columns: [SYMBOLS_BUY, TOKENS_BUY, Price]
Index: []

SYMBOLS_SELL  TOKENS_SELL    Price
0   BAJFINANCE        81153  3053.15


 BUY/SELL 15 minute frame


Empty DataFrame
Columns: [SYMBOLS_BUY, TOKENS_BUY, Price]
Index: []

...

SYMBOLS_SELL  TOKENS_SELL   Price
0          IOC       415745  147.35
1         ONGC       633601  143.70



  	 	 	 	 	 ALL ' BUY | SELL '  - Neglecting Common


BUY from ALL   set()
SELL From ALL  {81153, 415745, 633601}


  	 	 	 	 	 COMMON ' BUY & SELL ' 


BUY from ALL   set()
SELL From ALL  set()


BUY sell COMMON STOCK


Empty DataFrame
Columns: [BUY_common_symbol, BUY_common_Token]
Index: []

Empty DataFrame
Columns: [SELL_common_symbol, SELL_common_Token]
Index: []

	 	 	 	 	 !! ALL COMPLETED !!
 
 
